In [ ]:
#score: 0.97956
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

In [ ]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/gundata/train_data.npy")
X_test = np.load("/kaggle/input/gundata/test_data.npy")
y_train = pd.read_csv('/kaggle/input/gundata/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
shape1, shape2 = (441, 100)  
X_train = np.reshape(X_train, (-1, shape1, shape2,1))
X_test = np.reshape(X_test, (X_test.shape[0], shape1, shape2,1))

In [ ]:
print('Build LSTM RNN model ...')
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2],1)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
#model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()

In [ ]:

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 100, epochs = 100)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = model.predict(X_test)
#np.savetxt("y_pred_ann.csv", y_pred, delimiter=",")
  
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_conv_128_64_32_LSTM_64.csv", index=False)
